In [1]:
from utils import get_simple_powerpoint_info

In [2]:
get_simple_powerpoint_info()

{'파일 이름': 'TeXBLEU_ppt.pptx', '슬라이드 수': 21}

In [3]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
import anthropic
import os
from dotenv import load_dotenv

In [5]:
load_dotenv()
ANTHROPIC_API_KEY = os.environ.get('ANTHROPIC_API_KEY')
client = anthropic.Anthropic(
    
    api_key=ANTHROPIC_API_KEY,
)
message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)
print(message.content)


[TextBlock(citations=None, text="Hello! I'm Claude. How can I assist you today? I'm here to answer questions, discuss ideas, or help with various tasks. Feel free to let me know what you'd like to talk about.", type='text')]


In [6]:
message

Message(id='msg_019qbbRHBDaZtYn3riFawSdY', content=[TextBlock(citations=None, text="Hello! I'm Claude. How can I assist you today? I'm here to answer questions, discuss ideas, or help with various tasks. Feel free to let me know what you'd like to talk about.", type='text')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=10, output_tokens=46))

In [8]:
json_file = """
{'understanding': 'Translate the content of slide 2 to English.', 'tasks': [{'page number': 2, 'description': 'Translate all text elements on the slide', 'target': 'All text boxes, shapes with text, and title', 'action': 'Translate to English', 'contents': 'Found 6 objects in the slide number 2.\nObject 1:\n  Name: Title 1\n  Type: Placeholder\n  Position: Left=66.0, Top=28.75\n  Size: Width=828.0, Height=104.37503814697266\n  Text content: 전체적인 내용\n  Font: Arial, Size: 44.0\n  Bold: -1, Italic: 0\n  Alignment: Left\n  Cannot retrieve all text formatting details\nObject 2:\n  Name: Content Placeholder 2\n  Type: Placeholder\n  Position: Left=66.0, Top=133.1250457763672\n  Size: Width=639.142822265625, Height=84.3077163696289\n  Text content: 트랜스포머는 인코더와 디코더로 이루어져 있습니다.\n  Font: Arial, Size: 28.0\n  Bold: -1, Italic: 0\n  Alignment: Left\n  Cannot retrieve all text formatting details\nObject 3:\n  Name: Slide Number Placeholder 3\n  Type: Placeholder\n  Position: Left=678.0, Top=500.5\n  Size: Width=216.0, Height=28.75\n  Text content: 2 of 30\n  Font: Arial, Size: 18.0\n  Bold: 0, Italic: 0\n  Alignment: Right\n  Cannot retrieve all text formatting details\nObject 4:\n  Name: TextBox 5\n  Type: Text Box\n  Position: Left=66.0, Top=280.9012451171875\n  Size: Width=347.7142639160156, Height=75.12653350830078\n  Text content: 내 이름은 Kyudan Jung입니다.\n  Font: Arial, Size: 28.0\n  Bold: -1, Italic: 0\n  Alignment: Left\n  Cannot retrieve all text formatting details\nObject 5:\n  Name: Picture 6\n  Type: Picture\n  Position: Left=612.5714111328125, Top=253.15748596191406\n  Size: Width=245.6103973388672, Height=195.36520385742188\n  Picture: Picture 6\n  Alternative Text: \nObject 6:\n  Name: TextBox 8\n  Type: Text Box\n  Position: Left=66.0, Top=425.5000915527344\n  Size: Width=559.9999389648438, Height=46.0452766418457\n  Text content: 순수이성비판은 칸트의 저서입니다.\n  Font: Arial, Size: 32.0\n  Bold: -1, Italic: 0\n  Alignment: Left\n  Cannot retrieve all text formatting details\nParsing complete.', 'edit target type': ['Title 1', 'Content Placeholder 2', 'Slide Number Placeholder 3', 'TextBox 5', 'TextBox 8'], 'edit target content': ['전체적인 내용', '트랜스포머는 인코더와 디코더로 이루어져 있습니다.', '2 of 30', '내 이름은 Kyudan Jung입니다.', '순수이성비판은 칸트의 저서입니다.'], 'content after edit': ['Overall Content', 'Transformers consist of an encoder and a decoder.', '2 of 30', 'My name is Kyudan Jung.', 'Critique of Pure Reason is a book by Kant.']}]}
"""
from test_Applier import test_Applier
applier = test_Applier(api_key=ANTHROPIC_API_KEY)
applier(json_file)


True

In [27]:
ex_json = """

{'understanding': 'Translate the content of slide 2 into English.', 'tasks': [{'page number': 2, 'description': 'Translate all text elements on the slide', 'target': 'All text boxes, shapes with text, and title', 'action': 'Translate to English', 'contents': 'Found 6 objects in the slide number 2.\nObject 1:\n  Name: Title 1\n  Type: Placeholder\n  Position: Left=66.0, Top=28.75\n  Size: Width=828.0, Height=104.37503814697266\n  Text content: 전체적인 내용\n  Font: Arial, Size: 44.0\n  Bold: -1, Italic: 0\n  Alignment: Left\n  Cannot retrieve all text formatting details\nObject 2:\n  Name: Content Placeholder 2\n  Type: Placeholder\n  Position: Left=66.0, Top=133.1250457763672\n  Size: Width=639.142822265625, Height=84.3077163696289\n  Text content: 트랜스포머는 인코더와 디코더로 이루어져 있습니다.\n  Font: Arial, Size: 28.0\n  Bold: -1, Italic: 0\n  Alignment: Left\n  Cannot retrieve all text formatting details\nObject 3:\n  Name: Slide Number Placeholder 3\n  Type: Placeholder\n  Position: Left=678.0, Top=500.5\n  Size: Width=216.0, Height=28.75\n  Text content: 2 of 30\n  Font: Arial, Size: 18.0\n  Bold: 0, Italic: 0\n  Alignment: Right\n  Cannot retrieve all text formatting details\nObject 4:\n  Name: TextBox 5\n  Type: Text Box\n  Position: Left=66.0, Top=280.9012451171875\n  Size: Width=347.7142639160156, Height=75.12653350830078\n  Text content: 내 이름은 Kyudan Jung입니다.\n  Font: Arial, Size: 28.0\n  Bold: -1, Italic: 0\n  Alignment: Left\n  Cannot retrieve all text formatting details\nObject 5:\n  Name: Picture 6\n  Type: Picture\n  Position: Left=612.5714111328125, Top=253.15748596191406\n  Size: Width=245.6103973388672, Height=195.36520385742188\n  Picture: Picture 6\n  Alternative Text: \nObject 6:\n  Name: TextBox 8\n  Type: Text Box\n  Position: Left=66.0, Top=425.5000915527344\n  Size: Width=559.9999389648438, Height=46.0452766418457\n  Text content: 순수이성비판은 칸트의 저서입니다.\n  Font: Arial, Size: 32.0\n  Bold: -1, Italic: 0\n  Alignment: Left\n  Cannot retrieve all text formatting details\nParsing complete.', 'edit target type': ['Title 1', 'Content Placeholder 2', 'Slide Number Placeholder 3', 'TextBox 5', 'TextBox 8'], 'edit target content': ['전체적인 내용', '트랜스포머는 인코더와 디코더로 이루어져 있습니다.', '2 of 30', '내 이름은 Kyudan Jung입니다.', '순수이성비판은 칸트의 저서입니다.'], 'content after edit': ['Overall Content', 'Transformers consist of an encoder and a decoder.', '2 of 30', 'My name is Kyudan Jung.', 'Critique of Pure Reason is a book by Kant.']}]}
"""

In [28]:
import json
def load_json_with_direct_replace(json_str):
    # 모든 제어 문자를 직접 치환
    json_str = json_str.strip()
    
    # 1. 작은따옴표를 큰따옴표로 변환
    json_str = json_str.replace("'", '"')
    
    # 2. 모든 제어 문자를 이스케이프 처리
    # 줄바꿈, 탭, 캐리지 리턴 등을 이스케이프 시퀀스로 변환
    escaped_str = ""
    for c in json_str:
        if c == '\n':
            escaped_str += '\\n'
        elif c == '\t':
            escaped_str += '\\t'
        elif c == '\r':
            escaped_str += '\\r'
        elif ord(c) < 32:  # 기타 제어 문자
            escaped_str += f'\\u{ord(c):04x}'
        else:
            escaped_str += c
    
    # 3. JSON 파싱
    return json.loads(escaped_str)

In [29]:
processed_json=load_json_with_direct_replace(ex_json)

In [30]:
def _call_claude_api(prompt, api_key):
    import json
    import urllib.request
    import urllib.error

    
    # API 요청 준비
    url = "https://api.anthropic.com/v1/messages"
    headers = {
        "x-api-key": api_key,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    
    # API 요청 데이터
    data = {
        "model": "claude-3-7-sonnet-20250219",
        "messages": [
            {
                "role": "user",
                "content": f"{prompt}"
            }
        ],
        "max_tokens": 4000
    }

    
# 요청 설정
    req = urllib.request.Request(url)
    for key, value in headers.items():
        req.add_header(key, value)
    
    # JSON 데이터 인코딩
    json_data = json.dumps(data).encode('utf-8')
    
    # API 호출
    with urllib.request.urlopen(req, json_data) as response:
        response_text = response.read().decode('utf-8')
        result = json.loads(response_text)
        
        print("API 응답 수신 완료")
        return result["content"][0]["text"]
    

In [31]:
import json
def  _generate_code(api_key, type, before, after,
                                  slide_num, total_content):

    # API 호출을 위한 프롬프트 구성
    prompt = f"""
    Generate Python code to modify an active PowerPoint presentation based on the provided JSON task data.

    The code should:
    1. Find each slide specified by page number: {slide_num}
    2. Please refer to all of the following content: {total_content}
    3. What you need to change is {type} and the previous content is {before}.
    4. Finally, make the modification as {after}.
    5. Create executable code, and only generate code.

    Remember:
- Use only the win32com library for PowerPoint interactions.
- Generate executable code only, with no additional text or explanations.
- Include necessary error handling and logging within the script.
- Ensure the script is complete and can be run as-is, assuming the PowerPoint application is open with an active presentation.

    """

        
    # Claude API 호출
    code = _call_claude_api(prompt, api_key)
    
    # 코드 전처리
    code = code.strip()
    if code.startswith("```python"):
        code = code[len("```python"):].strip()
    if code.startswith("```"):
        code = code[3:].strip()
    if code.endswith("```"):
        code = code[:-3].strip()
    
    # error_occurred 변수 확인
    if "error_occurred = False" not in code:
        code = "error_occurred = False\n\n" + code
    
    return code

In [39]:
load_dotenv()
ANTHROPIC_API_KEY = os.environ.get('ANTHROPIC_API_KEY')
for task_idx, task in enumerate(processed_json['tasks']):
    print(task)
    slide_num = task['page number']
    total_content = task['contents']
    
    edit_target_type = task['edit target type']
    edit_target_content = task['edit target content']
    content_after_edit = task['content after edit']

    for type, before, after in zip(edit_target_type, edit_target_content, content_after_edit):
        if before == after:
            print("")
        else:
            code = _generate_code(ANTHROPIC_API_KEY, type, before, after,
                                  slide_num, total_content)
            exec(code)
            

{'page number': 2, 'description': 'Translate all text elements on the slide', 'target': 'All text boxes, shapes with text, and title', 'action': 'Translate to English', 'contents': 'Found 6 objects in the slide number 2.\nObject 1:\n  Name: Title 1\n  Type: Placeholder\n  Position: Left=66.0, Top=28.75\n  Size: Width=828.0, Height=104.37503814697266\n  Text content: 전체적인 내용\n  Font: Arial, Size: 44.0\n  Bold: -1, Italic: 0\n  Alignment: Left\n  Cannot retrieve all text formatting details\nObject 2:\n  Name: Content Placeholder 2\n  Type: Placeholder\n  Position: Left=66.0, Top=133.1250457763672\n  Size: Width=639.142822265625, Height=84.3077163696289\n  Text content: 트랜스포머는 인코더와 디코더로 이루어져 있습니다.\n  Font: Arial, Size: 28.0\n  Bold: -1, Italic: 0\n  Alignment: Left\n  Cannot retrieve all text formatting details\nObject 3:\n  Name: Slide Number Placeholder 3\n  Type: Placeholder\n  Position: Left=678.0, Top=500.5\n  Size: Width=216.0, Height=28.75\n  Text content: 2 of 30\n  Font: Arial, S